In [33]:
import openai
import os

API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-03-15-preview"


In [34]:
itinerary_prompt = [
    {
        'role': 'system',
        'content': 'The system is an AI assistant that helps people find information.'
    },
    {
        'role': 'system',
        'content': '\nGiven a json document describing a trip provided by a user, generate a day-by-day trip itinerary to satisfy those parameters.\n\nWhen the trip mentions a recommended place to visit (for example: a hotel, a landmark / point of interest, or a restaurant), it should be a specific location.\n\nWhenever the name of a place to visit (a hotel, a landmark / point of interest, or a restaurant) is mentioned, it must be in the following format:\n[LOCATION_NAME_ENTITY|{"name": "<name of location>", "country": "<country name>", "latitude": <latitude>, "longitude": <longitude>}]\n\nFor example:\n\n[Seattle|{"name": "Seattle", "country": "United States", "latitude": 47.6062, "longitude": -122.3321}]\n[Miami|{"name": "Miami", "country": "United States", "latitude": 25.7617, "longitude": -80.1918}]\n[Nashville|{"name": "Nashville", "country": "United States", "latitude": 36.1627, "longitude": -86.7816}]\n[The Ritz-Carlton, Boston|{"name": "The Ritz-Carlton", "country": "United States", "latitude": 42.3505, "longitude": -71.0735}]\n[The Westin Copley Place, Boston|{"name": "The Westin Copley Place", "country": "United States", "latitude": 42.3496, "longitude": -71.0763}]\n[The Langham, Boston|{"name": "The Langham", "country": "United States", "latitude": 42.3494, "longitude": -71.0784}]\n[Fenway Park|{"name": "Fenway Park", "country": "United States", "latitude": 42.3466, "longitude": -71.0972}]\n[Museum of Fine Arts, Boston|{"name": "Museum of Fine Arts", "country": "United States", "latitude": 42.3394, "longitude": -71.0942}]\n[Boston Tea Party Ships & Museum|{"name": "Boston Tea Party Ships & Museum", "country": "United States", "latitude": 42.3553, "longitude": -71.0572}]\n[Oleana|{"name": "Oleana", "country": "United States", "latitude": 42.3746, "longitude": -71.1019}]\n[Neptune Oyster|{"name": "Neptune Oyster", "country": "United States", "latitude": 42.3632, "longitude": -71.0545}]\n[Little Donkey|{"name": "Little Donkey", "country": "United States", "latitude": 42.3641, "longitude": -71.1012}]\n\n\nA trip itinerary must follow the following format:\n\n{\n    "title": <short title describing the type of trip and kinds of places visited>,\n    "description": <500 character or fewer description of the trip, explaining why it\'s relevant to the description provided by the user>\n    "days": [\n        <Day 1: text description of the first day of the trip>,\n        <Day 2: text description of the second day of the trip>,\n        ...\n    ]\n}\n\n\nThe assistant can only respond in JSON format, using the above structure.\n'},
    {
        'role': 'user',
        'content': '\n    {\n        "destination": "Chicago, Illinois, United States",\n        "number of days": 2,\n        "traveling with": "family - with young children",\n        "time of year": "September",\n        "interested in": ["history", "architecture"]\n    } \n    '
    }
]

In [35]:
%%timeit

response = openai.ChatCompletion.create(
    engine="gpt-4",
    model="gpt-4",
    messages=itinerary_prompt,
)

36.8 s ± 7.01 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
response["choices"][0]["message"]["content"]

'{\n    "Field Museum": "At the Field Museum, explore history, science, and art exhibits, including the famous T. Rex skeleton called Sue. While some areas may not be suitable for small children, upper floor gems and jade exhibits offer a quieter experience. Plan ahead for parking, ticket purchasing, and possibly bringing your own food to avoid high museum cafe prices. The museum\'s size can provide multiple visit-worthy learning experiences and discoveries.",\n    "Grant Park": "Relax and let the kids play in the easily accessible Grant Park, with great city views, particularly from the water. Visit the iconic Buckingham Fountain and nearby Lake Michigan. Although seating may be limited, the park is safe and perfect for leisure walks. Explore nearby locations like The Bean in Millennium Park and enjoy events hosted in Grant Park, an urban oasis.",\n    "The Art Institute of Chicago": "Immerse yourselves in the vast collection of art at The Art Institute of Chicago, featuring works fro

In [37]:
response["usage"]

<OpenAIObject at 0x7f201bb41a90> JSON: {
  "completion_tokens": 343,
  "prompt_tokens": 1780,
  "total_tokens": 2123
}

In [38]:
itinerary_context = """
TripAdvisor URL to cite: https://www.tripadvisor.com/ShowTopic-g35805-i32-k14023874 Facts:

    Stay in River North instead of the Downtown/Loop area as it is safer and closer to attractions.
    Use Spothero.com or Parkwhiz.com to pre-book discounted parking at a garage near your hotel.
    Opt for a lot without "in and out privileges" to save money on parking.
    Use feet, public transit and/or cabs and Uber to get around while exploring the city.
    Visit either the 360 Observation Deck (in the building formally known as the Hancock Tower) or the Skydeck at Willis Tower, but not both.
    River cruises during the day are better.
    Lincoln Park Zoo may not be a "don't miss" activity for adults unless you have a particular interest in animals, especially primates.
    The Planetarium would not make my Top 10 list unless you have a special interest in astronomy.
    Wait until the day of your planned visit to buy Willis Tower or Hancock tickets to avoid bad visibility.
    Pre-book some dinners to avoid difficulty making reservations on the weekend.
    Rent a bike or electric bike for a beautiful short ride along the lake.
    Use ParkWhiz or Spothero to locate and save money on nearby parking.

TripAdvisor URL to cite: https://www.tripadvisor.com/ShowTopic-g35805-i32-k14094345 Facts:

    Use Uber or Lyft to get around if you're staying near O'Hare and want to go downtown.
    Consider a CTA 1-day pass for $5 if you're planning to use public transportation frequently.
    The Chicago Architecture Center river cruise is highly recommended, but if you want to save money, Wendella Cruise is a good alternative.
    Giordano's is known for stuffed pizza, not deep dish. Pizzano's and Lou Malnati's are good options for deep dish.
    Use Google Maps for tracking buses and trains, or consider using the G-Pay app to pay for transportation.
    If you want to visit non-downtown areas, Logan Square, Wicker Park or Bucktown are good options.
    Choose Chicago website has many lists of things to do in the city.
    When visiting Millennium Park, make sure to check out "the Bean" sculpture (formal name is "Cloudgate"), the Crown Fountain, and the Lurie Garden.
    If you want to visit a jazz bar, Andy's Jazz and Jazz Showcase are good options downtown.
    Take caution when taking the Blue Line back to Rosemont after early evening.
    The Illusions Museum is a for-profit chain of museums operated by franchisees, so it may not be worth visiting if you have a limited amount of time in the city.
    The GpsMyCity app has walking tours for the downtown area.
    Navy Pier and the Museum of Science and Industry are both worth visiting, but you may not have time to do both.
    The Green Mill jazz bar is best to visit via Uber or Lyft due to its location in a not-so-safe area at night.
    Check out "Sunday on State" for live music and entertainment.

TripAdvisor URL to cite: https://www.tripadvisor.com/ShowTopic-g35805-i32-k13978255 Facts:

    Be aware of spamming and fake reviews on travel forums.
    Use reputable sources like choosechicago.com to find comprehensive lists of things to do in a city.
    Don't be afraid to ask questions or for recommendations from locals or friends who have visited the city before.
    Research attractions before visiting to make sure they align with your interests and values.
    Stay open-minded but also trust your gut instincts when it comes to deciding which attractions to visit.

TripAdvisor URL to cite: https://www.tripadvisor.com/ShowTopic-g35805-i32-k14308633 Facts:

    If you arrive at the airport early, be prepared for most of the shops to still be closed.
    The weather forecasting in Chicago is not always accurate, so be prepared for unexpected changes.
    If you're visiting Chicago during St. Patrick's Day, make sure to catch the river dying green.
    Be aware of incidental charges that hotels may charge, which can be a deposit for any damage to the room or stolen property.
    Make sure to double-check your food order before leaving the restaurant.
    When visiting Billy Goat Tavern, expect a dive-bar-like atmosphere and very thin burger patties.
    If you're planning to visit the Field Museum, make sure to plan out your itinerary thoroughly.
    Take the opportunity to ride the Brown line train to see various neighborhoods and unique architecture.
    Always be prepared for cold weather in Chicago and dress in layers.
    If you're looking for authentic Italian beef sandwiches, Portillo's is a recommended spot.
    Be careful of the messy gravy on the dipped sandwich at Portillo's.
    Stan's Donuts has a Bailey Irish Cream donut that is heavenly.
    Make sure to check for any missing items in your food order at restaurants.

TripAdvisor URL to cite: https://www.tripadvisor.com/ShowTopic-g35805-i32-k13969374 Facts:

    It's best to keep outdoor plans open in Chicago due to the variable weather.
    Making reservations for popular restaurants in advance is recommended, especially on weekends.
    Wendella offers flexible cruise options, with the last sailing at 9pm on Wednesdays.
    The Riverwalk and Art on the Mart are worth checking out.
    Millennium Park offers free concerts at night and has a few outdoor bars to visit for a drink afterwards.
    The Chicago Cultural Center is located across the street from Millennium Park and is free to visit.
    StubHub is a good option for last-minute Cubs tickets.
    Kingston Mines is a popular blues club located a little over a mile south of Wrigley.
    River North Hotel (Acme) is a good accommodation option.
    The city is generally safe to take public transportation during the day, but better to rely on taxis/lyfts at night.
"""

itinerary_prompt_with_context = [
    {
        'role': 'system',
        'content': 'The system is an AI assistant that helps people find information.'
    },
    {
        'role': 'system',
        'content': 'Given the following context from TripAdvisor forums: ' + itinerary_context + '\n\nGiven a json document describing a trip provided by a user, generate a day-by-day trip itinerary to satisfy those parameters.\n\nWhen the trip mentions a recommended place to visit (for example: a hotel, a landmark / point of interest, or a restaurant), it should be a specific location.\n\nWhenever the name of a place to visit (a hotel, a landmark / point of interest, or a restaurant) is mentioned, it must be in the following format:\n[LOCATION_NAME_ENTITY|{"name": "<name of location>", "country": "<country name>", "latitude": <latitude>, "longitude": <longitude>}]\n\nFor example:\n\n[Seattle|{"name": "Seattle", "country": "United States", "latitude": 47.6062, "longitude": -122.3321}]\n[Miami|{"name": "Miami", "country": "United States", "latitude": 25.7617, "longitude": -80.1918}]\n[Nashville|{"name": "Nashville", "country": "United States", "latitude": 36.1627, "longitude": -86.7816}]\n[The Ritz-Carlton, Boston|{"name": "The Ritz-Carlton", "country": "United States", "latitude": 42.3505, "longitude": -71.0735}]\n[The Westin Copley Place, Boston|{"name": "The Westin Copley Place", "country": "United States", "latitude": 42.3496, "longitude": -71.0763}]\n[The Langham, Boston|{"name": "The Langham", "country": "United States", "latitude": 42.3494, "longitude": -71.0784}]\n[Fenway Park|{"name": "Fenway Park", "country": "United States", "latitude": 42.3466, "longitude": -71.0972}]\n[Museum of Fine Arts, Boston|{"name": "Museum of Fine Arts", "country": "United States", "latitude": 42.3394, "longitude": -71.0942}]\n[Boston Tea Party Ships & Museum|{"name": "Boston Tea Party Ships & Museum", "country": "United States", "latitude": 42.3553, "longitude": -71.0572}]\n[Oleana|{"name": "Oleana", "country": "United States", "latitude": 42.3746, "longitude": -71.1019}]\n[Neptune Oyster|{"name": "Neptune Oyster", "country": "United States", "latitude": 42.3632, "longitude": -71.0545}]\n[Little Donkey|{"name": "Little Donkey", "country": "United States", "latitude": 42.3641, "longitude": -71.1012}]\n\n\nA trip itinerary must follow the following format:\n\n{\n    "title": <short title describing the type of trip and kinds of places visited>,\n    "description": <500 character or fewer description of the trip, explaining why it\'s relevant to the description provided by the user>\n    "days": [\n        <Day 1: text description of the first day of the trip>,\n        <Day 2: text description of the second day of the trip>,\n        ...\n    ]\n}\n\n\nThe assistant can only respond in JSON format, using the above structure.\n'},
    {
        'role': 'user',
        'content': '\n    {\n        "destination": "Chicago, Illinois, United States",\n        "number of days": 10,\n        "traveling with": "family - with young children",\n        "time of year": "September",\n        "interested in": ["history", "architecture"]\n    } \n    '
    }
]

In [39]:
%%timeit

response = openai.ChatCompletion.create(
    engine="gpt-4",
    model="gpt-4",
    messages=itinerary_prompt_with_context,
)

1min 11s ± 10.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
response["usage"]

<OpenAIObject at 0x7f201bb41a90> JSON: {
  "completion_tokens": 343,
  "prompt_tokens": 1780,
  "total_tokens": 2123
}

In [41]:
response["choices"][0]["message"]["content"]

'{\n    "Field Museum": "At the Field Museum, explore history, science, and art exhibits, including the famous T. Rex skeleton called Sue. While some areas may not be suitable for small children, upper floor gems and jade exhibits offer a quieter experience. Plan ahead for parking, ticket purchasing, and possibly bringing your own food to avoid high museum cafe prices. The museum\'s size can provide multiple visit-worthy learning experiences and discoveries.",\n    "Grant Park": "Relax and let the kids play in the easily accessible Grant Park, with great city views, particularly from the water. Visit the iconic Buckingham Fountain and nearby Lake Michigan. Although seating may be limited, the park is safe and perfect for leisure walks. Explore nearby locations like The Bean in Millennium Park and enjoy events hosted in Grant Park, an urban oasis.",\n    "The Art Institute of Chicago": "Immerse yourselves in the vast collection of art at The Art Institute of Chicago, featuring works fro

In [42]:
location_description_prompt = [
    {
        'role': 'system',
        'content': 'The system is an AI assistant that helps people find information.'
    },
    {
        'role': 'system',
        'content': '\nGiven an ordered list of comma separated target locations (<location1>, <location2>, ...), generate a 100 word description for each location.\n\nTry to explain how each location may be of particular interest to the user accounting for:\n- the user\'s interests, if any\n- who the user is traveling with, if provided\n- how each location may fit well thematically or by proximity into the day\'s itinerary, i.e., with other locations, for the user\n\nAnswer in JSON format, using the following structure:\n{\n    "location1": <100 word description of location1>,\n    "location2": <100 word description of location2>,\n    ...\n}\n'
    },
    {
        'role': 'user',
        'content': "\n    Target locations: Field Museum, Grant Park, The Art Institute of Chicago, Giordano's\n    Day itinerary: Day 1: Start your day with a visit to the Field Museum to explore history, science, and art. Afterward, take a stroll to Grant Park for some relaxation and let the kids play. In the afternoon, visit The Art Institute of Chicago to appreciate incredible architecture and fine art. Finally, dine at Giordano's to taste Chicago's famous deep-dish pizza.\n    User interests: history, architecture\n    Traveling with: family with young children\n    "
    }
]

In [43]:
%%timeit

response = openai.ChatCompletion.create(
    engine="gpt-4",
    model="gpt-4",
    messages=location_description_prompt,
)

14.1 s ± 4.32 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
response["choices"][0]["message"]["content"]

'{\n    "Field Museum": "At the Field Museum, explore history, science, and art exhibits, including the famous T. Rex skeleton called Sue. While some areas may not be suitable for small children, upper floor gems and jade exhibits offer a quieter experience. Plan ahead for parking, ticket purchasing, and possibly bringing your own food to avoid high museum cafe prices. The museum\'s size can provide multiple visit-worthy learning experiences and discoveries.",\n    "Grant Park": "Relax and let the kids play in the easily accessible Grant Park, with great city views, particularly from the water. Visit the iconic Buckingham Fountain and nearby Lake Michigan. Although seating may be limited, the park is safe and perfect for leisure walks. Explore nearby locations like The Bean in Millennium Park and enjoy events hosted in Grant Park, an urban oasis.",\n    "The Art Institute of Chicago": "Immerse yourselves in the vast collection of art at The Art Institute of Chicago, featuring works fro

In [45]:
response["usage"]

<OpenAIObject at 0x7f201bb41a90> JSON: {
  "completion_tokens": 343,
  "prompt_tokens": 1780,
  "total_tokens": 2123
}

In [46]:
location_description_context = '\nUse the following tips, advice, and facts about each location (if provided) above to generate the description:\n\nField Museum: 1. The Field Museum can take more than a day to explore due to its size and scope; plan accordingly. 2. It might not be suitable for small children, as there are limited interactive exhibits and lots of reading. 3. The special exhibits may require an additional fee but can be worth it depending on your interests. 4. Some areas, like the Egyptian exhibit, have low lighting, making it challenging to read information. 5. Food inside the museum can be expensive; consider bringing your own food, as it is allowed. 6. The Field Museum is famous for its T. Rex skeleton called Sue and the large dinosaur exhibit, which is a must-see. 7. Visit the jade and gems exhibits on the upper floor, which are usually less crowded in the morning. 8. Using public transportation to get to the museum is highly recommended instead of driving and dealing with parking. 9. Handicap accessible East Entrance (on the path towards Shedd Aquarium) can have shorter lines. 10. When planning a visit, check the Field Museum\'s website to see if it coincides with a free admission day, which can result in large crowds and longer wait times. 11. Use a City Pass for potential access to special ticketed exhibits and a 3D movie, though these extras are not considered essential for a great visit. 12. The 3D movie about Sue the T. Rex is highly rated, but the theater can be crowded. 13. The museum cafe\'s food quality and options might not meet everyone\'s preferences; consider bringing your own meal or snacks. 14. Purchase tickets ahead of time, as they sell out fast during peak times or due to the pandemic-entry restrictions. 15. The museum features exhibits on natural history, evolution, animal species, plants, gems, and human history. 16. Each visit to the museum can provide new learning experiences and discoveries, making it worthwhile to visit multiple times.\nGrant Park: 1. Grant Park is easy to find and provides a relaxing environment for leisurely walks. 2. The park offers great views of the city, and it\'s recommended to see it from the water for the best experience. 3. Buckingham Fountain is a must-see attraction within the park. 4. Grant Park is located across the street from Lake Michigan, where there are walking and biking paths. 5. The park may lack benches and shaded areas, so visitors should be prepared for limited seating options. 6. In addition to its foliage, Grant Park has iconic sculptures worth seeing. 7. The park hosts outdoor concerts and events, though parts may be closed off during festival preparations. 8. Grant Park is easily accessible from downtown and near the museum campus at the southern end. 9. The park is well-designed, maintained, and offers a great greenspace within the city. 10. Visitors can enjoy fall colors and feel safe in the well-patrolled park. 11. Be prepared for long walks, as it\'s easy to lose track of time exploring the area. 12. Recommended stops nearby include The Bean in Millennium Park, Robert\'s pizza, and Navy Pier. 13. Limited seating is available in most parts of the park, which may be inconvenient for resting. 14. Grant Park is part of a series of parks, starting from Millennium Park and stretching to the Field Museum. 15. The park is safe and frequented by people walking dogs and engaging in other outdoor activities. 16. There are shaded and non-shaded areas, so be prepared for varying temperatures during summer visits. 17. Grant Park is about a 15-minute walk from Millennium Park, and is considered an "urban oasis." 18. The park is located near Lake Michigan, offering additional recreational opportunities.\nThe Art Institute of Chicago: 1. The Art Institute of Chicago has a vast collection of art, including African, Asian, American, Impressionist, Modern, and Contemporary pieces. 2. It showcases works by renowned artists such as Monet, Renoir, Seurat, Morisot, Degas, Van Gogh, Dali, Picasso, and more. 3. Visitors recommend spending at least 3-5 hours or even multiple days to fully experience the museum\'s collections. 4. For a shorter visit, use the museum\'s highlights guide for what to see in an hour. 5. Special exhibits, like the Cezanne exhibit, may have an additional entrance fee. 6. The museum offers a free tour at 12 pm on certain days. 7. The building\'s architecture, including stairs and long walks between wings, may be challenging for people with mobility issues. 8. Accessibility may be improved with an electric wheelchair. 9. The checkroom may have long wait times during peak periods, so factor in at least 30 minutes for picking up items. 10. All cafes and snack spots are currently closed inside the museum. 11. The museum uses small font on descriptive signage and may have limited docents available for questions. 12. Backpacks are not allowed inside the museum, but photography without flash is permitted. 13. Consider using the museum\'s app or asking about tours for a more interactive experience. 14. The museum shop is worth visiting on its own. 15. Consider purchasing a Citypass for discounted admission. 16. Arrive early for a less crowded experience and to maximize your time at the museum. 17. Plan your visit in advance with the help of a detailed map to navigate through the various exhibits efficiently.\'\n1. Giordano\'s is famous for its deep-dish pizza, considered by some as the best in the world. 2. Expect the restaurant to be busy, and be prepared to wait for a table. 3. You can pre-order your pizza while waiting for your table to speed up the process once seated. 4. A normal pizza costs around $25, with smaller pizzas having 6 slices; most people can eat around 2 slices. 5. The pizzas are quite large, so consider sharing your pizza or expect to take leftovers home. 6. The deep-dish pizza takes about 45 minutes to cook, though wait times may be longer in some cases. 7. The restaurant offers other menu items such as the Giordano\'s Sampler, which includes boneless chicken nuggets, mozzarella triangles, and other appetizers to enjoy while waiting for your pizza. 8. Some visitors noted issues with staffing and delays, however, the management team is often gracious and helpful in addressing any concerns. 9. Giordano\'s hosts events such as rehearsal dinners, offering food and beverage packages at reasonable prices. 10. This particular location is conveniently situated near the Mag Mile shopping area, making it a great place for shoppers to take a break and grab a bite. 11. Other dishes, such as the BBQ wings, are also highly recommended by customers. 12. For authentic Chicago-style deep-dish pizza, Giordano\'s is considered an essential stop, comparable to Portillo\'s for Italian beef and hot dogs.\n'

location_description_prompt_with_context = [
    {
        'role': 'system',
        'content': 'The system is an AI assistant that helps people find information.'
    },
    {
        'role': 'system',
        'content': '\nGiven an ordered list of comma separated target locations (<location1>, <location2>, ...), generate a 100 word description for each location.\n\nTry to explain how each location may be of particular interest to the user accounting for:\n- the user\'s interests, if any\n- who the user is traveling with, if provided\n- how each location may fit well thematically or by proximity into the day\'s itinerary, i.e., with other locations, for the user\n\nAnswer in JSON format, using the following structure:\n{\n    "location1": <100 word description of location1>,\n    "location2": <100 word description of location2>,\n    ...\n}\n'
    },
    {
        'role': 'user',
        'content': "\n    Target locations: Field Museum, Grant Park, The Art Institute of Chicago, Giordano's\n    Day itinerary: Day 1: Start your day with a visit to the Field Museum to explore history, science, and art. Afterward, take a stroll to Grant Park for some relaxation and let the kids play. In the afternoon, visit The Art Institute of Chicago to appreciate incredible architecture and fine art. Finally, dine at Giordano's to taste Chicago's famous deep-dish pizza.\n    User interests: history, architecture\n    Traveling with: family with young children\n    "
    },
    {
        'role': 'user',
        'content': location_description_context
    }
]

In [47]:
%%timeit

response = openai.ChatCompletion.create(
    engine="gpt-4",
    model="gpt-4",
    messages=location_description_prompt_with_context,
)

18.5 s ± 4.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
response["usage"]

<OpenAIObject at 0x7f201bb41a90> JSON: {
  "completion_tokens": 343,
  "prompt_tokens": 1780,
  "total_tokens": 2123
}

In [49]:
response["choices"][0]["message"]["content"]

'{\n    "Field Museum": "At the Field Museum, explore history, science, and art exhibits, including the famous T. Rex skeleton called Sue. While some areas may not be suitable for small children, upper floor gems and jade exhibits offer a quieter experience. Plan ahead for parking, ticket purchasing, and possibly bringing your own food to avoid high museum cafe prices. The museum\'s size can provide multiple visit-worthy learning experiences and discoveries.",\n    "Grant Park": "Relax and let the kids play in the easily accessible Grant Park, with great city views, particularly from the water. Visit the iconic Buckingham Fountain and nearby Lake Michigan. Although seating may be limited, the park is safe and perfect for leisure walks. Explore nearby locations like The Bean in Millennium Park and enjoy events hosted in Grant Park, an urban oasis.",\n    "The Art Institute of Chicago": "Immerse yourselves in the vast collection of art at The Art Institute of Chicago, featuring works fro